第七章: 数据清洗和准备
在进行数据分析和建模的过程中，大量的时间花在数据准备上：加载、清理、转换和重新排列。
在本章中，我将讨论用于缺失值、重复值、字符串操作和其他数据转换的工具。下一章中，我将重点关注利用各种方法对数据集联合、重排列。

7.1 处理缺失值
对于数值型数据，pandas 使用浮点值NAN。我们称NAN为容易检测到的标识值。

In [1]:
import pandas as pd
import numpy as np 

In [3]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
print(string_data)
print(string_data.isnull())

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object
0    False
1    False
2     True
3    False
dtype: bool


在pandas 中，我们采用了R语言中的编程惯例，将缺失值成为NA，意思是not available(不可用)。在统计学中，NA数据可以是不存在的数据或者是存在但不可以观察的数据（例如在数据收集过程中出现了问题）。当清洗数据用于分析时，对缺数据本身进行分析以确定数据收集问题或数据丢失导致数据偏差通常很重要。

In [8]:
string_data[0] = None
print(string_data.isnull())  # 返回表明哪些值是缺失值的布尔值
print(string_data.dropna())  # 根据每个标签的值是否缺失数据来筛选轴标签，并允许丢失的数据量来确定阈值
print(string_data.fillna('hello'))  # 用某些值填充缺失的数据或用插值方法（如'ffill'或'bfill'）
print(string_data.notnull())  #  isnull 的反函数

0     True
1    False
2     True
3    False
dtype: bool
1    artichoke
3      avocado
dtype: object
0        hello
1    artichoke
2        hello
3      avocado
dtype: object
0    False
1     True
2    False
3     True
dtype: bool


7.1.1 过滤缺失值


In [11]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [12]:
# 上面的代码等价于
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [14]:
# dropna 默认情况下会删除包含缺失值的行
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan], [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])
clean = data.dropna()
print(data)
print(clean)

     0    1    2
0  1.0  6.5  3.0
1  1.0  NaN  NaN
2  NaN  NaN  NaN
3  NaN  6.5  3.0
     0    1    2
0  1.0  6.5  3.0


In [16]:
# 传入how='all' 时，将删除所有值均为NA的行
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [18]:
# 如果要用同样的方式去删除列，传入参数axis=1
data[4] = np.nan
print(data)
data.dropna(axis=1, how='all')

     0    1    2   4
0  1.0  6.5  3.0 NaN
1  1.0  NaN  NaN NaN
2  NaN  NaN  NaN NaN
3  NaN  6.5  3.0 NaN


,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [22]:
# 过滤DataFrame 的行的相关方法往往涉及时间序列数据。假设你只想保留包含一定数量的观察值的行。你可以用thresh参数来表示
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
print(df)

print(df.dropna())
print(df.dropna(thresh=2))

          0         1         2
0 -0.671558       NaN       NaN
1 -2.072681       NaN       NaN
2 -0.465074       NaN  0.236121
3  0.429361       NaN  0.518240
4 -0.463601  0.808191  2.123445
5 -1.806658  0.408206  0.795206
6 -1.484296 -0.601941 -1.127979
          0         1         2
4 -0.463601  0.808191  2.123445
5 -1.806658  0.408206  0.795206
6 -1.484296 -0.601941 -1.127979
          0         1         2
2 -0.465074       NaN  0.236121
3  0.429361       NaN  0.518240
4 -0.463601  0.808191  2.123445
5 -1.806658  0.408206  0.795206
6 -1.484296 -0.601941 -1.127979


7.1.2 补全缺失值

In [23]:
# 使用fillna方法来补全缺失值
df.fillna(0)

,0,1,2
0,-0.671558,0.000000,0.000000
1,-2.072681,0.000000,0.000000
2,-0.465074,0.000000,0.236121
3,0.429361,0.000000,0.518240
4,-0.463601,0.808191,2.123445
5,-1.806658,0.408206,0.795206
6,-1.484296,-0.601941,-1.127979


In [24]:
# 在调用fillna时使用字典，你可以为不同列设定不同的填充值
df.fillna({1:0.5, 2:0})

,0,1,2
0,-0.671558,0.500000,0.000000
1,-2.072681,0.500000,0.000000
2,-0.465074,0.500000,0.236121
3,0.429361,0.500000,0.518240
4,-0.463601,0.808191,2.123445
5,-1.806658,0.408206,0.795206
6,-1.484296,-0.601941,-1.127979


In [25]:
# fillna返回的是一个新对象，但你也可以修改已经存在的对象
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-0.671558,0.000000,0.000000
1,-2.072681,0.000000,0.000000
2,-0.465074,0.000000,0.236121
3,0.429361,0.000000,0.518240
4,-0.463601,0.808191,2.123445
5,-1.806658,0.408206,0.795206
6,-1.484296,-0.601941,-1.127979


In [29]:
# 用于重建索引的相同的插值方法也可以用于fillna
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
print(df)

'''
    fillna函数参数
        value:标量值或字典型对象用于填充缺失值
        method:插值方法，如果没有其他参数，默认是ffill
        axis:需要填充的轴，默认axis=0
        inplace:修改被调用的对象，而不是生成一个备份
        limit:用于前向或后向填充时最大的填充范围
'''
print(df.fillna(method='ffill'))
print(df.fillna(method='ffill', limit=2))

          0         1         2
0  0.115105  0.011041  0.210894
1 -1.802301  1.001189 -0.829573
2 -1.068148       NaN -1.939378
3 -1.698171       NaN  0.405176
4 -1.357074       NaN       NaN
5  0.928057       NaN       NaN
          0         1         2
0  0.115105  0.011041  0.210894
1 -1.802301  1.001189 -0.829573
2 -1.068148  1.001189 -1.939378
3 -1.698171  1.001189  0.405176
4 -1.357074  1.001189  0.405176
5  0.928057  1.001189  0.405176
          0         1         2
0  0.115105  0.011041  0.210894
1 -1.802301  1.001189 -0.829573
2 -1.068148  1.001189 -1.939378
3 -1.698171  1.001189  0.405176
4 -1.357074       NaN  0.405176
5  0.928057       NaN  0.405176


7.2 数据转换

7.2.1 删除重复值

In [31]:
# 由于各种原因，DataFrame中会出现重复行。
data = pd.DataFrame({'k1':['one', 'two']*3+['two'], 'k2':[1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [32]:
# DataFrame 的duplicates 方法返回的是一个布尔值Series,这个Series反映的是每一行是否存在重复（与之前出现过的行相同）情况
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [33]:
# drop_duplicates返回的是DataFrame,内容是duplicated返回数组中为False的部分：
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [35]:
# 这些方法默认都是对列进行操作。你可以指定数据的任何子集来检测是否有重复。假设我们有一个额外的列，并想基于‘k1’列去除重复值
data['v1'] = range(7)
print(data)
data.drop_duplicates(['k1'])

    k1  k2  v1
0  one   1   0
1  two   1   1
2  one   2   2
3  two   3   3
4  one   3   4
5  two   4   5
6  two   4   6


,k1,k2,v1
0,one,1,0
1,two,1,1


In [36]:
# drop_duplicates 和 duplicated 默认都是保留第一个观测到的值。传入参数keep='last'将会返回最后一个
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


In [ ]:
7.2.2 使用函数或映射进行数据转换
